### 데이터베이스 접속, 테이블 만들기 

In [ ]:
import pymysql
db = pymysql.connect(host='localhost',port=3306,user='root',passwd='Code6121!',charset='utf8')

cursor = db.cursor()
sql = 'drop database if exists GmarKet'
cursor.execute(sql)
db.commit()
db.close


In [ ]:
import pymysql
db = pymysql.connect(host='localhost',port=3306,user='root',passwd='Code6121!',charset='utf8')

cursor = db.cursor()
sql = 'create database gmarket'
cursor.execute(sql)
db.commit()
db.close


In [ ]:
import pymysql
db = pymysql.connect(host='localhost',port=3306,user='root',passwd='Code6121!',db='gmarket',charset='utf8')
cursor = db.cursor()

sql = '''
CREATE TABLE items(
    item_code VARCHAR(20) NOT NULL PRIMARY KEY,
    title VARCHAR(200) NOT NULL,
    ori_price INT NOT NULL,
    dis_price INT NOT NULL,
    dis_percent INT NOT NULL,
    provider VARCHAR(100)
);
'''
cursor.execute(sql)

sql = '''
CREATE TABLE ranking(
    num INT AUTO_INCREMENT NOT NULL PRIMARY KEY,
    main_category VARCHAR(50) NOT NULL,
    sub_category VARCHAR(50) NOT NULL,
    item_ranking TINYINT UNSIGNED NOT NULL,
    item_code VARCHAR(20) NOT NULL,
    FOREIGN KEY (item_code) REFERENCES items(item_code)
);
'''
cursor.execute(sql)

db.commit()
db.close()



###  크롤링 : main category 정보 가져오기

In [ ]:
import requests 
from bs4 import BeautifulSoup

res = requests.get('http://corners.gmarket.co.kr/Bestsellers')
soup = BeautifulSoup(res.content, 'html.parser')

categories = soup.select('div.gbest-cate ul.by-group li > a')
for category in categories:
    print('http://corners.gmarket.co.kr/' + category['href'], category.get_text())




### 크롤링 : main/sub category 정보 가져오기

In [ ]:
import requests 
from bs4 import BeautifulSoup

res = requests.get('http://corners.gmarket.co.kr/Bestsellers')
soup = BeautifulSoup(res.content, 'html.parser')

categories = soup.select('div.gbest-cate ul.by-group li > a')
for category in categories:
    get_category('http://corners.gmarket.co.kr/' + category['href'], category.get_text())



In [ ]:
def get_category(category_link,category_name):
    print(category_link, category_name)
    res = requests.get(category_link)
    soup = BeautifulSoup(res.content, 'html.parser')
    
    sub_categories = soup.select('div.navi.group ul li > a')
    for sub_category in sub_categories:
        print(category_link, category_name,'http://corners.gmarket.co.kr/' + sub_category['href'], sub_category.get_text())


###  main/sub category + 상품 정보 크롤링

In [ ]:
import requests
from bs4 import BeautifulSoup
import pymysql
db = pymysql.connect(host='localhost', port=3306, user='root', passwd='Code6121!', db='gmarket', charset='utf8')
cursor = db.cursor()

res = requests.get('http://corners.gmarket.co.kr/Bestsellers')
soup = BeautifulSoup(res.content, 'html.parser')

categories = soup.select('div.gbest-cate ul.by-group li > a')
for category in categories:
    get_category('http://corners.gmarket.co.kr/' + category['href'], category.get_text())



In [ ]:
def get_category(category_link,category_name):
    print(category_link, category_name)
    res = requests.get(category_link)
    soup = BeautifulSoup(res.content, 'html.parser')
    
    get_items(soup, category_name, "ALL") # ALL 메인카테고리에서 전체
    
    sub_categories = soup.select('div.navi.group ul li > a')
    for sub_category in sub_categories:
        print(category_link, category_name,'http://corners.gmarket.co.kr/' + sub_category['href'], sub_category.get_text())


In [ ]:
def get_items(html, category_name, sub_category_name): # 앞서 이미 파싱되어있어->html
    items_result_list = list()
    best_item = html.select('div.best-list')
    for index, item in enumerate(best_item[1].select('li')):  # 페이지 소스에 best-list가 3개 나오는데 거기서 두번째 값-> 인덱스[1]
        ranking = index + 1
        title = item.select_one('a.itemname') # .get_text() 객체가 없는상태에서 호출하려고하면 에러가 난다
        ori_price = item.select_one('div.o-price span')                         
        dis_price = item.select_one('div.s-price strong span')
        dis_percent = item.select_one('div.s-price span > em')
        
        if ori_price == None or ori_price.get_text() == '': # 두번째는 태그는 있으나 안에 값이 없을경우 // 순서를 바꿀경우 NULL인 상태에서 get.text()할경우 에런발생
            ori_price = dis_price
            
        if dis_price == None: # 할인이 없을경우 정가가 dis_price위치에 들어가서 NULL인 경우 정가 할인가가 0원
            ori_price, dis_price = 0, 0
        
        else:
            ori_price = ori_price.get_text().replace(',','').replace('원','') # 15,900원 -> 15900
            dis_price = dis_price.get_text().replace(',','').replace('원','')
        
        if dis_percent == None or dis_percent.get_text() == '':
            dis_percent = 0
        
        else:
            dis_percent = dis_percent.get_text().replace('%','')
        
        print(category_name, sub_category_name, ranking, title.get_text(), ori_price, dis_price, dis_percent) # 타이틀만 get.text()를 한다 나머지는 if문에서 get.text()를 한상태
                                

### main/sub category + 상품 정보  크롤링_2

In [ ]:
import requests
from bs4 import BeautifulSoup
import pymysql
db = pymysql.connect(host='localhost', port=3306, user='root', passwd='Code6121!', db='gmarket', charset='utf8') # mysql 접속하기
cursor = db.cursor() # 커서 가져오기 # 데이터베이스 커서는 일련의 데이터에 순차적으로 엑세스할 때 검색 및 "현재 위치"를 포함하는 데이터 요소

res = requests.get('http://corners.gmarket.co.kr/Bestsellers') # 웹페이지 가져오기
soup = BeautifulSoup(res.content, 'html.parser') # 웹페이지 파싱하기(문자열의 의미 분석)

categories = soup.select('div.gbest-cate ul.by-group li > a') # 필요한 데이터 추출
for category in categories: # 메인 카테고리를 크롤링
    get_category('http://corners.gmarket.co.kr/' + category['href'], category.get_text()) # 메인카테고리+서브카테고리+상품 정보 

In [ ]:
def get_category(category_link,category_name): # get_카테고리 함수 만들기(카테고리 링크,이름)
    print(category_link, category_name)
    res = requests.get(category_link)
    soup = BeautifulSoup(res.content, 'html.parser')
    
    get_items(soup, category_name, "ALL") # ALL 메인카테고리에서 전체
    
    sub_categories = soup.select('div.navi.group ul li > a')
    for sub_category in sub_categories: # 서브 카테고리를 크롤링
        res = requests.get('http://corners.gmarket.co.kr/' + sub_category['href'])
        soup = BeautifulSoup(res.content, 'html.parser')
        get_items(soup, category_name, sub_category.get_text()) # 서브카테고리와 서브카테고리에 있는 상품정보 호출 
        

In [ ]:
def get_items(html, category_name, sub_category_name): # 앞서 이미 파싱되어있어->html
    items_result_list = list()
    best_item = html.select('div.best-list')
    for index, item in enumerate(best_item[1].select('li')):  # 페이지 소스에 best-list가 3개 나오는데 거기서 두번째 값-> 인덱스[1]
        ranking = index + 1
        title = item.select_one('a.itemname') # .get_text() 객체가 없는상태에서 호출하려고하면 에러가 난다
        ori_price = item.select_one('div.o-price span')                         
        dis_price = item.select_one('div.s-price strong span')
        dis_percent = item.select_one('div.s-price span > em')
        
        if ori_price == None or ori_price.get_text() == '': # 두번째는 태그는 있으나 안에 값이 없을경우 // 순서를 바꿀경우 NULL인 상태에서 get.text()할경우 에런발생
            ori_price = dis_price
            
        if dis_price == None: # 할인이 없을경우 정가가 dis_price위치에 들어가서 NULL인 경우 정가 할인가가 0원
            ori_price, dis_price = 0, 0
        
        else:
            ori_price = ori_price.get_text().replace(',','').replace('원','') # 15,900원 -> 15900
            dis_price = dis_price.get_text().replace(',','').replace('원','')
        
        if dis_percent == None or dis_percent.get_text() == '':
            dis_percent = 0
        
        else:
            dis_percent = dis_percent.get_text().replace('%','')
        
        print(category_name, sub_category_name, ranking, title.get_text(), ori_price, dis_price, dis_percent)
               

### main/sub category + 상품 정보 + 상품코드 + 판매자  크롤링_3

In [ ]:
import requests
from bs4 import BeautifulSoup
import pymysql
db = pymysql.connect(host='localhost', port=3306, user='root', passwd='Code6121!', db='gmarket', charset='utf8')
cursor = db.cursor()

res = requests.get('http://corners.gmarket.co.kr/Bestsellers')
soup = BeautifulSoup(res.content, 'html.parser')

categories = soup.select('div.gbest-cate ul.by-group li > a') # 메인 카테고리 데이터 추출
for category in categories:
    get_category('http://corners.gmarket.co.kr/' + category['href'], category.get_text())



In [ ]:
def get_category(category_link,category_name):
    print(category_link, category_name)
    res = requests.get(category_link)
    soup = BeautifulSoup(res.content, 'html.parser')
    
    get_items(soup, category_name, "ALL") # ALL 메인카테고리에서 전체
    
    sub_categories = soup.select('div.navi.group ul li > a') # 서브 카테고리 데이터 추출
    for sub_category in sub_categories: # 서브카테고리를 크롤링
        res = requests.get('http://corners.gmarket.co.kr/' + sub_category['href'])
        soup = BeautifulSoup(res.content, 'html.parser')
        get_items(soup, category_name, sub_category.get_text()) # 서브카테고리에 있는 상품정보 호출 
        



In [ ]:
def get_items(html, category_name, sub_category_name): # 앞서 이미 파싱되어있어->html
    items_result_list = list()
    best_item = html.select('div.best-list') # 아이템 추출
    for index, item in enumerate(best_item[1].select('li')):  # 페이지 소스에 best-list가 3개 나오는데 거기서 두번째 값-> 인덱스[1]
        ranking = index + 1
        title = item.select_one('a.itemname') # .get_text()를 객체가 없는상태에서 호출하려고하면 에러가 난다
        ori_price = item.select_one('div.o-price span')                         
        dis_price = item.select_one('div.s-price strong span')
        dis_percent = item.select_one('div.s-price span > em')
        
        if ori_price == None or ori_price.get_text() == '': # 두번째는 태그는 있으나 안에 값이 없을경우 // 순서를 바꿀경우 NULL인 상태에서 get.text()할경우 에런발생
            ori_price = dis_price
            
        if dis_price == None: # 할인이 없을경우 정가가 dis_price위치에 들어가서 NULL인 경우 정가 할인가가 0원
            ori_price, dis_price = 0, 0
        
        else:
            ori_price = ori_price.get_text().replace(',','').replace('원','') # 15,900원 -> 15900
            dis_price = dis_price.get_text().replace(',','').replace('원','')
        
        if dis_percent == None or dis_percent.get_text() == '':
            dis_percent = 0
        
        else:
            dis_percent = dis_percent.get_text().replace('%','')
            
        product_link = item.select_one('div.thumb > a') # 제품 링크 추출
        item_code = product_link.attrs['href'].split('=')[1] # attrs 속성값 모두 출력, 제품 링크에서 상품코드 추출
        
        res = requests.get(product_link.attrs['href']) # 상품링크 가져오기
        soup = BeautifulSoup(res.content, 'html.parser')
        provider = soup.select_one('div.item-topinfo_headline > p > span > a') # 판매자 데이터 추출
# select_one해야 오류가 안난다
# find_all 혹은 select로 얻어진 전체 데이터중에서 find or select_one으로 단위 데이터를 뺴내기 위해서는 for 순환문을 사용합니다.
# find와 select 참고링크 https://smorning.tistory.com/331
        if provider == None:
            provider = ''
        else:
            provider = provider.get_text()
        
        print(category_name, sub_category_name, ranking, item_code, provider, title.get_text(), ori_price, dis_price, dis_percent)

            
            
        


#### main/sub category + 상품 정보 + 상품코드 + 판매자 (딕셔너리형태로 바꿈)

In [ ]:
import requests
from bs4 import BeautifulSoup
import pymysql
db = pymysql.connect(host='localhost', port=3306, user='root', passwd='Code6121!', db='gmarket', charset='utf8')
cursor = db.cursor()

res = requests.get('http://corners.gmarket.co.kr/Bestsellers')
soup = BeautifulSoup(res.content, 'html.parser')

categories = soup.select('div.gbest-cate ul.by-group li > a') # 메인 카테고리 데이터 추출
for category in categories:
    get_category('http://corners.gmarket.co.kr/' + category['href'], category.get_text())



In [ ]:
def get_category(category_link,category_name):
    print(category_link, category_name)
    res = requests.get(category_link)
    soup = BeautifulSoup(res.content, 'html.parser')
    
    get_items(soup, category_name, "ALL") # ALL 메인카테고리에서 전체
    
    sub_categories = soup.select('div.navi.group ul li > a') # 서브 카테고리 데이터 추출
    for sub_category in sub_categories: # 서브카테고리를 크롤링
        res = requests.get('http://corners.gmarket.co.kr/' + sub_category['href'])
        soup = BeautifulSoup(res.content, 'html.parser')
        get_items(soup, category_name, sub_category.get_text()) # 서브카테고리에 있는 상품정보 호출 
        


In [ ]:
def get_items(html, category_name, sub_category_name): # 앞서 이미 파싱되어있어->html
    items_result_list = list()
    best_item = html.select('div.best-list') # 아이템 추출
    for index, item in enumerate(best_item[1].select('li')):  # 페이지 소스에 best-list가 3개 나오는데 거기서 두번째 값-> 인덱스[1]
        data_dict = dict() # 딕셔너리로 만듬 key:value
        ranking = index + 1
        title = item.select_one('a.itemname') # .get_text()를 객체가 없는상태에서 호출하려고하면 에러가 난다
        ori_price = item.select_one('div.o-price span')                         
        dis_price = item.select_one('div.s-price strong span')
        dis_percent = item.select_one('div.s-price span > em')
        
        if ori_price == None or ori_price.get_text() == '': # 두번째는 태그는 있으나 안에 값이 없을경우 // 순서를 바꿀경우 NULL인 상태에서 get.text()할경우 에런발생
            ori_price = dis_price
            
        if dis_price == None: # 할인이 없을경우 정가가 dis_price위치에 들어가서 NULL인 경우 정가 할인가가 0원
            ori_price, dis_price = 0, 0
        
        else:
            ori_price = ori_price.get_text().replace(',','').replace('원','') # 15,900원 -> 15900
            dis_price = dis_price.get_text().replace(',','').replace('원','')
        
        if dis_percent == None or dis_percent.get_text() == '':
            dis_percent = 0
        
        else:
            dis_percent = dis_percent.get_text().replace('%','')
            
        product_link = item.select_one('div.thumb > a') # 제품 링크 추출
        item_code = product_link.attrs['href'].split('=')[1] # attrs 속성값 모두 출력, 제품 링크에서 상품코드 추출
        
        res = requests.get(product_link.attrs['href']) # 상품링크 가져오기
        soup = BeautifulSoup(res.content, 'html.parser')
        provider = soup.select_one('div.item-topinfo_headline > p > span > a') # 판매자 데이터 추출
# select_one해야 오류가 안난다
# find_all 혹은 select로 얻어진 전체 데이터중에서 find or select_one으로 단위 데이터를 뺴내기 위해서는 for 순환문을 사용합니다.
# find와 select 참고링크 https://smorning.tistory.com/331
        if provider == None:
            provider = ''
        else:
            provider = provider.get_text()
            
        data_dict['category_name'] = category_name
        data_dict['sub_category_name'] = sub_category_name
        data_dict['ranking'] = ranking
        data_dict['title'] = title.get_text()
        data_dict['ori_price'] = ori_price
        data_dict['dis_price'] = dis_price
        data_dict['dis_percent'] = dis_percent
        data_dict['item_code'] = item_code
        data_dict['provider'] = provider  
        
        print(data_dict)
        
#         print(category_name, sub_category_name, ranking, item_code, provider, title.get_text(), ori_price, dis_price, dis_percent)

            
            
        


### INSERT SQL 만들기 

##### Table 구조
```sql
CREATE TABLE ranking (
    num INT AUTO_INCREMENT NOT NULL PRIMARY KEY,
    main_category VARCHAR(20) NOT NULL,
    sub_category VARCHAR(20) NOT NULL,
    item_ranking TINYINT NOT NULL,
    item_code VARCHAR(10) NOT NULL,
    FOREIGN KEY (item_code) REFERENCES items(item_code)
);
```

In [ ]:
item_info={'category_name': 'ALL', 
           'sub_category_name': 'ALL', 
           'ranking': 1, 
           'title': '[배스킨라빈스](배스킨라빈스) 파티 위드 트윙클 트리', 
           'ori_price': '29000', 
           'dis_price': '22620', 
           'dis_percent': '22', 
           'item_code': '2300771272&ver', 
           'provider': '맛있는 해피콘'}

In [ ]:
sql = """INSERT INTO ranking (main_category, sub_category, item_ranking, item_code) VALUES(
'""" + item_info['category_name'] + """',
'""" + item_info['sub_category_name'] + """', 
'""" + str(item_info['ranking']) + """', 
'""" + item_info['item_code'] + """')"""      


In [ ]:
sql.replace('\n','')

#### Table 구조
```sql
CREATE TABLE items (
    item_code VARCHAR(10) NOT NULL PRIMARY KEY,
    title VARCHAR(200) NOT NULL,
    ori_price INT NOT NULL,
    dis_price INT NOT NULL,
    discount_percent INT NOT NULL,
    provider VARCHAR(100)
);
```

In [ ]:
sql = """INSERT INTO items VALUES(
'""" + item_info['item_code'] + """',
'""" + item_info['title'] + """', 
""" + item_info['ori_price'] + """, 
""" + item_info['dis_price'] + """, 
""" + item_info['dis_percent'] + """, 
'""" + item_info['provider'] + """')"""    

In [ ]:
sql.replace('\n','')

### 목적
- 실제로 데이터베이스를 사용하는 방법을 현업스타일로 설명을 드리는 것
- 현실 세계의 데이터를 어떻게 테이블로 정의할 것인가
- 어떻게 데이터를 넣을 것인가? (프로그래밍으로 넣는 것이 일반적)
- 저장된 데이터를 분석한다. (SQL만 쓰는 경우 + 프로그래밍에서 데이터를 분석해서, 풀스택으로 서비스화함)

##### 기존 SQL 강의: 단순히 세세한 SQL 문법을 다 소개받고, 끝

In [3]:
import pymysql
db = pymysql.connect(host='localhost', port=3306, user='root', passwd='Code6121!', db='gmarket', charset='utf8')
cursor = db.cursor()

sql = "DELETE FROM ranking"
cursor.execute(sql)

sql = "DELETE FROM items"
cursor.execute(sql)

db.commit()
db.close()

# DELETE FROM 테이블이름 WHERE 필드이름=데이터값 delete문은 해당 테이블에서 where절의 조건을 만족하는 레코드만을 삭제한다. where절을 생략하면 해당 테이블에 저장된 모든 데이터가 삭제된다.

#### COUNT SQL 
- COUNT: 검색 결과의 row 수를 가져올 수 있는 SQL 문법
- SQL 예제: SELECT COUNT(*) FROM items

```sql
sql = """SELECT COUNT(*) FROM items WHERE item_code = 
'""" + item_info['item_code'] + """';"""
cursor.execute(sql)
result = cursor.fetchone() 하나의 레코드를 가져온다
print (result[0]) 그 레코드에서 인덱스0의 값
```

- COUNT SQL 예제1: SELECT COUNT(*) FROM items (전체 row 수)
- COUNT SQL 예제2: SELECT COUNT(ori_price) FROM items (ori_price field 값이 있는 row 수)

```
SELECT COUNT(*) FROM items WHRER item_code = 123124324
저 숫자에 해당되는 아이템코드가 있으면 1, 없으면 0
```

In [ ]:
sql = """SELECT COUNT(*) FROM items WHERE item_code = '""" + item_info['item_code'] + """';"""

In [ ]:
sql

In [ ]:
def save_data(item_info):
    print(item_info)
    sql = """SELECT COUNT(*) FROM items WHERE item_code = '""" + item_info['item_code'] + """';"""
    cursor.execute(sql)
    result = cursor.fetchone() # 하나의 레코드를 가져온다
    print (result[0]) 
    if result[0] == 0: # 저 아이템코드에 해당되는 아이템이 없을때(여러 카테고리에 있을 아이템 중복방지) 
        sql = """INSERT INTO items VALUES(
        '""" + item_info['item_code'] + """',
        '""" + item_info['title'] + """', 
        """ + str(item_info['ori_price']) + """, 
        """ + str(item_info['dis_price']) + """, 
        """ + str(item_info['dis_percent']) + """, 
        '""" + item_info['provider'] + """')"""
        print(sql)
        cursor.execute(sql)
    
    sql = """INSERT INTO ranking (main_category, sub_category, item_ranking, item_code) VALUES(
    '""" + item_info['category_name'] + """',
    '""" + item_info['sub_category_name'] + """', 
    '""" + str(item_info['ranking']) + """', 
    '""" + item_info['item_code'] + """')"""     
    print(sql)
    cursor.execute(sql)
    


## 마지막

In [ ]:
import requests
from bs4 import BeautifulSoup
import pymysql
db = pymysql.connect(host='localhost', port=3306, user='root', passwd='Code6121!', db='gmarket', charset='utf8')
cursor = db.cursor() # 커서 가져오기 # 데이터베이스 커서는 일련의 데이터에 순차적으로 엑세스할 때 검색 및 "현재 위치"를 포함하는 데이터 요소

res = requests.get('http://corners.gmarket.co.kr/Bestsellers') # 웹페이지 가져오기
soup = BeautifulSoup(res.content, 'html.parser') # 웹페이지 파싱하기(문자열의 의미 분석)

categories = soup.select('div.gbest-cate ul.by-group li > a') # 메인 카테고리 데이터 추출
for category in categories: # 메인 카테고리를 크롤링
    get_category('http://corners.gmarket.co.kr/' + category['href'], category.get_text()) # 메인카테고리+서브카테고리


In [ ]:
def get_category(category_link,category_name): # 겟_카테고리 함수 만들기(카테고리 링크, 이름)
    print(category_link, category_name)
    res = requests.get(category_link)
    soup = BeautifulSoup(res.content, 'html.parser')
    
    get_items(soup, category_name, "ALL") # ALL 메인카테고리에서 전체
    
    sub_categories = soup.select('div.navi.group ul li > a') # 서브 카테고리 데이터 추출
    for sub_category in sub_categories: # 서브카테고리를 크롤링
        res = requests.get('http://corners.gmarket.co.kr/' + sub_category['href'])
        soup = BeautifulSoup(res.content, 'html.parser')
        get_items(soup, category_name, sub_category.get_text()) # 서브카테고리에 있는 상품정보 호출 
        

In [ ]:
def get_items(html, category_name, sub_category_name): # 앞서 이미 파싱되어있어->html
    items_result_list = list()
    best_item = html.select('div.best-list') # 아이템 추출
    for index, item in enumerate(best_item[1].select('li')):  # 페이지 소스에 best-list가 3개 나오는데 거기서 두번째 값-> 인덱스[1]
        data_dict = dict() # 딕셔너리로 만듬 key:value
        ranking = index + 1
        title = item.select_one('a.itemname') # .get_text()를 객체가 없는상태에서 호출하려고하면 에러가 난다
        ori_price = item.select_one('div.o-price span')                         
        dis_price = item.select_one('div.s-price strong span')
        dis_percent = item.select_one('div.s-price span > em')
        
        if ori_price == None or ori_price.get_text() == '': # 두번째는 태그는 있으나 안에 값이 없을경우 // 순서를 바꿀경우 NULL인 상태에서 get.text()할경우 에런발생
            ori_price = dis_price
            
        if dis_price == None: # 할인이 없을경우 정가가 dis_price위치에 들어가서 NULL인 경우 정가 할인가가 0원
            ori_price, dis_price = 0, 0
        
        else:
            ori_price = ori_price.get_text().replace(',','').replace('원','') # 15,900원 -> 15900
            dis_price = dis_price.get_text().replace(',','').replace('원','')
        
        if dis_percent == None or dis_percent.get_text() == '':
            dis_percent = 0
        
        else:
            dis_percent = dis_percent.get_text().replace('%','')
            
        product_link = item.select_one('div.thumb > a') # 제품 링크 추출
        item_code = product_link.attrs['href'].split('=')[1] # attrs 속성값 모두 출력, 제품 링크에서 상품코드 추출
        
        res = requests.get(product_link.attrs['href']) # 상품링크 가져오기
        soup = BeautifulSoup(res.content, 'html.parser')
        provider = soup.select_one('div.item-topinfo_headline > p > span > a') # 판매자 데이터 추출
# select_one해야 오류가 안난다
# find_all 혹은 select로 얻어진 전체 데이터중에서 find or select_one으로 단위 데이터를 뺴내기 위해서는 for 순환문을 사용합니다.
# find와 select 참고링크 https://smorning.tistory.com/331
        if provider == None:
            provider = ''
        else:
            provider = provider.get_text()
# 딕셔너리           
        data_dict['category_name'] = category_name
        data_dict['sub_category_name'] = sub_category_name
        data_dict['ranking'] = ranking
        data_dict['title'] = title.get_text()
        data_dict['ori_price'] = ori_price
        data_dict['dis_price'] = dis_price
        data_dict['dis_percent'] = dis_percent
        data_dict['item_code'] = item_code
        data_dict['provider'] = provider  
        
        save_data(data_dict)

In [ ]:
def save_data(item_info): # item_info -> data_dict
    print(item_info)
    sql = """SELECT COUNT(*) FROM items WHERE item_code = '""" + item_info['item_code'] + """';"""
    cursor.execute(sql) # sql 실행
    result = cursor.fetchone() # 하나의 레코드를 가져온다
    print (result[0]) # 인덱스 0의 값
    if result[0] == 0: # 1이 되면 아이템이 중복되어 데이터베이스에 아이템을 추가 저장하지 않는다 
        # 저 아이템코드에 해당되는 아이템이 없을때(여러 카테고리에 있을 아이템 중복방지) 
        # 0이 아이템이 중복되진 않는 다는걸 보여줌
        sql = """INSERT INTO items VALUES(
        '""" + item_info['item_code'] + """',
        '""" + item_info['title'] + """', 
        """ + str(item_info['ori_price']) + """, 
        """ + str(item_info['dis_price']) + """, 
        """ + str(item_info['dis_percent']) + """, 
        '""" + item_info['provider'] + """')"""
#         print(sql)
        cursor.execute(sql)
    
    sql = """INSERT INTO ranking (main_category, sub_category, item_ranking, item_code) VALUES(
    '""" + item_info['category_name'] + """',
    '""" + item_info['sub_category_name'] + """', 
    '""" + str(item_info['ranking']) + """', 
    '""" + item_info['item_code'] + """')"""     
#     print(sql)
    cursor.execute(sql)

## 최종

In [1]:
import requests
from bs4 import BeautifulSoup
import pymysql

def save_data(item_info): # item_info -> data_dict
    print(item_info)
    sql = """SELECT COUNT(*) FROM items WHERE item_code = '""" + item_info['item_code'] + """';"""
    cursor.execute(sql) # sql 실행
    result = cursor.fetchone() # 하나의 레코드를 가져온다
    print (result[0])  # 인덱스 0의 값 
    if result[0] == 0: # 1이 되면 아이템이 중복되어 데이터베이스에 아이템을 추가 저장하지 않는다 
        # 저 아이템코드에 해당되는 아이템이 없을때(여러 카테고리에 있을 아이템 중복방지) 
        # 0이 아이템이 중복되진 않는 다는걸 보여줌
        sql = """INSERT INTO items VALUES(
        '""" + item_info['item_code'] + """',
        '""" + item_info['title'] + """', 
        """ + str(item_info['ori_price']) + """, 
        """ + str(item_info['dis_price']) + """, 
        """ + str(item_info['dis_percent']) + """, 
        '""" + item_info['provider'] + """')"""
#         print(sql)
        cursor.execute(sql)
    
    sql = """INSERT INTO ranking (main_category, sub_category, item_ranking, item_code) VALUES(
    '""" + item_info['category_name'] + """',
    '""" + item_info['sub_category_name'] + """', 
    '""" + str(item_info['ranking']) + """', 
    '""" + item_info['item_code'] + """')"""     
#     print(sql)
    cursor.execute(sql)
    
def get_items(html, category_name, sub_category_name): # 앞서 이미 파싱되어있어->html
    items_result_list = list()
    best_item = html.select('div.best-list') # 아이템 추출
    for index, item in enumerate(best_item[1].select('li')):  # 페이지 소스에 best-list가 3개 나오는데 거기서 두번째 값-> 인덱스[1]
        data_dict = dict() # 딕셔너리로 만듬 key:value
        ranking = index + 1
        title = item.select_one('a.itemname') # .get_text()를 객체가 없는상태에서 호출하려고하면 에러가 난다
        ori_price = item.select_one('div.o-price span')                         
        dis_price = item.select_one('div.s-price strong span')
        dis_percent = item.select_one('div.s-price span > em')
        
        if ori_price == None or ori_price.get_text() == '': # 두번째는 태그는 있으나 안에 값이 없을경우 // 순서를 바꿀경우 NULL인 상태에서 get.text()할경우 에런발생
            ori_price = dis_price
            
        if dis_price == None: # 할인이 없을경우 정가가 dis_price위치에 들어가서 NULL인 경우 정가 할인가가 0원
            ori_price, dis_price = 0, 0
        
        else:
            ori_price = ori_price.get_text().replace(',','').replace('원','') # 15,900원 -> 15900
            dis_price = dis_price.get_text().replace(',','').replace('원','')
        
        if dis_percent == None or dis_percent.get_text() == '':
            dis_percent = 0
        
        else:
            dis_percent = dis_percent.get_text().replace('%','')
            
        product_link = item.select_one('div.thumb > a') # 제품 링크 추출
        item_code = product_link.attrs['href'].split('=')[1] # attrs 속성값 모두 출력, 제품 링크에서 상품코드 추출
        
        res = requests.get(product_link.attrs['href']) # 상품링크 가져오기
        soup = BeautifulSoup(res.content, 'html.parser')
        provider = soup.select_one('div.item-topinfo_headline > p > span > a') # 판매자 데이터 추출
# select_one해야 오류가 안난다
# find_all 혹은 select로 얻어진 전체 데이터중에서 find or select_one으로 단위 데이터를 뺴내기 위해서는 for 순환문을 사용합니다.
# find와 select 참고링크 https://smorning.tistory.com/331
        if provider == None:
            provider = ''
        else:
            provider = provider.get_text()
            
        data_dict['category_name'] = category_name
        data_dict['sub_category_name'] = sub_category_name
        data_dict['ranking'] = ranking
        data_dict['title'] = title.get_text()
        data_dict['ori_price'] = ori_price
        data_dict['dis_price'] = dis_price
        data_dict['dis_percent'] = dis_percent
        data_dict['item_code'] = item_code
        data_dict['provider'] = provider  
        
        save_data(data_dict)
        
def get_category(category_link,category_name): # 겟_카테고리 함수 만들기(카테고리 링크, 이름)
    print(category_link, category_name)
    res = requests.get(category_link)
    soup = BeautifulSoup(res.content, 'html.parser')
    
    get_items(soup, category_name, "ALL") # ALL 메인카테고리에서 전체
    
    sub_categories = soup.select('div.navi.group ul li > a') # 서브 카테고리 데이터 추출
    for sub_category in sub_categories: # 서브카테고리를 크롤링
        res = requests.get('http://corners.gmarket.co.kr/' + sub_category['href'])
        soup = BeautifulSoup(res.content, 'html.parser')
        get_items(soup, category_name, sub_category.get_text()) # 서브카테고리에 있는 상품정보 호출 

db = pymysql.connect(host='localhost', port=3306, user='root', passwd='Code6121!', db='gmarket', charset='utf8')
cursor = db.cursor() # 커서 가져오기 # 데이터베이스 커서는 일련의 데이터에 순차적으로 엑세스할 때 검색 및 "현재 위치"를 포함하는 데이터 요소

res = requests.get('http://corners.gmarket.co.kr/Bestsellers') # 웹페이지 가져오기
soup = BeautifulSoup(res.content, 'html.parser') # 웹페이지 파싱하기(문자열의 의미 분석)

categories = soup.select('div.gbest-cate ul.by-group li > a') # 메인 카테고리 데이터 추출
for category in categories: # 메인 카테고리를 크롤링
    get_category('http://corners.gmarket.co.kr/' + category['href'], category.get_text()) 

db.commit() # DB에 Complete 하기
db.close() # DB 연결 닫기


http://corners.gmarket.co.kr//Bestsellers ALL
{'category_name': 'ALL', 'sub_category_name': 'ALL', 'ranking': 1, 'title': '[배스킨라빈스](배스킨라빈스) 파티 위드 트윙클 트리', 'ori_price': '29000', 'dis_price': '22620', 'dis_percent': '22', 'item_code': '2300771272&ver', 'provider': '맛있는 해피콘'}
0
{'category_name': 'ALL', 'sub_category_name': 'ALL', 'ranking': 2, 'title': '[배스킨라빈스](배스킨라빈스) 파티 위드 스노우 볼', 'ori_price': '30000', 'dis_price': '23400', 'dis_percent': '22', 'item_code': '2300770251&ver', 'provider': '맛있는 해피콘'}
0
{'category_name': 'ALL', 'sub_category_name': 'ALL', 'ranking': 3, 'title': '(21년산쌀/수향미/상)  독정 수향미 10kg', 'ori_price': '37900', 'dis_price': '35630', 'dis_percent': '5', 'item_code': '1817548571&ver', 'provider': '롯데상사(주)안성공장'}
0
{'category_name': 'ALL', 'sub_category_name': 'ALL', 'ranking': 4, 'title': '[배스킨라빈스](배스킨라빈스) 파티 위드 해피박스', 'ori_price': '31000', 'dis_price': '24180', 'dis_percent': '22', 'item_code': '2300771945&ver', 'provider': '맛있는 해피콘'}
0
{'category_name': 'ALL', 'sub_categor

KeyboardInterrupt: 